In [1]:
from gensim import downloader

In [2]:
# all models
print(list(downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [3]:
word2vec_google_news_300 = downloader.load("word2vec-google-news-300")

In [4]:
import random

In [5]:
def guessWord(candidates):
    guess = random.randint(0, len(candidates) - 1)
    return candidates[guess]

In [6]:
question_words = []
answer_words = []
guess_words = []
is_guess = []

with open("synonym_test_dataset.txt", "r") as file:
    lineNo = 0
    for line in file:
        lineNo = lineNo + 1
        # new word
        if(lineNo % 6 == 1):
            word = line.split()[1]
            question_words.append(word)
            if word not in word2vec_google_news_300.key_to_index.keys():
                print(word + "(query word) is absent from model")
                guess = True;
            else:
                guess = False;
            candidates = []
            candidate_similarity = 0
            best_candidate = ""
            candidate_guess = 0
        # answer
        elif(lineNo % 6 == 0):
            if guess or candidate_guess == 4:
                guess_words.append(guessWord(candidates))
                is_guess.append(True)
            else:
                guess_words.append(best_candidate)
                is_guess.append(False)
            if line == 'a\n' or line == 'a':
                answer_words.append(candidates[0])
            elif line == 'b\n' or line == 'b':
                answer_words.append(candidates[1])
            elif line == 'c\n' or line == 'c':
                answer_words.append(candidates[2])
            elif line == 'd\n' or line == 'd':
                answer_words.append(candidates[3])
        # candidate synonyms
        else:
            candidate = line.split()[1]
            candidates.append(candidate)
            if candidate not in word2vec_google_news_300.key_to_index.keys():
                print(candidate + "(synonym candidate) is absent from model")
                candidate_guess = candidate_guess + 1
            elif not guess:
                similarity = word2vec_google_news_300.similarity(candidate, question_words[-1])
                if similarity > candidate_similarity:
                    candidate_similarity = similarity
                    best_candidate = candidate

model_name = "word2vec-google-news-300"
size_vocab = len(word2vec_google_news_300.key_to_index)
correct_label = 0
no_guessing = 80
with open(model_name + "-details.csv", "w+") as file:
    for i in range(0, len(question_words)):
        line = question_words[i] + "," + answer_words[i] + "," + guess_words[i] + ","
        if(is_guess[i]):
            line = line + "guess"
            no_guessing = no_guessing - 1
        elif(answer_words[i] == guess_words[i]):
            line = line + "correct"
            correct_label = correct_label + 1
        elif(answer_words[i] != guess_words[i]):
            line = line + "wrong"
        file.write(line + "\n")
        
with open("analysis.csv", "w+") as file:
    line = model_name + "," + str(size_vocab) + "," + str(correct_label) + "," + str(no_guessing) + "," + str(correct_label/no_guessing)
    file.write(line + "\n")

tranquillity(query word) is absent from model


In [8]:
import gc

In [9]:
# free memory
del word2vec_google_news_300
gc.collect()

136